In [17]:
import pandas as pd
import os

In [21]:
df = pd.read_csv(os.path.join("..", "output", "mtuberculosis", "assays_cleaned.csv"))
len(df[df['cpds'] >= 100])

69

In [22]:
a = df[['assay_id', 'activity_type', 'unit']].values
a = [tuple(i) for i in a]
len(set(a))

11347

In [23]:
a = df[['assay_id']].values
a = [tuple(i) for i in a]
len(a), len(set(a))

(11347, 10902)